In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
client_id = '72654f91e8424dc8b4924dc4cdf3674a'
client_secret='968caf037e674bb0a89533fde2831f25'

In [3]:
auth_manager = SpotifyClientCredentials(client_id = client_id,
                                       client_secret = client_secret)

In [4]:
sp = spotipy.Spotify(auth_manager=auth_manager)

In [16]:
def playlist_features(playlist_id):
    json = sp.playlist_tracks(playlist_id = playlist_id)
    json_items = json['items']
    df_normalized = pd.json_normalize(json_items)
    lista_tracks_ids = list(df_normalized['track.id'])
    json_features = sp.audio_features(tracks = lista_tracks_ids)
    df_features = pd.DataFrame(json_features)
    df = pd.merge(left = df_normalized,
                  right = df_features,
                  left_on = 'track.id',
                  right_on = 'id')
    return df

In [17]:
df_top_2020 = playlist_features(playlist_id = '37i9dQZF1DXdJSo3JjzjFs')

In [23]:
top_ids_per_year = ['37i9dQZF1DX7Jl5KP2eZaS',
                    '37i9dQZF1DWVRSukIED0e9',
                    '37i9dQZF1DXe2bobNYDtW8',
                    '37i9dQZF1DWTE7dVUebpUW',
                    '37i9dQZF1DX8XZ6AUo9R4R',
                    '37i9dQZF1DX9ukdrXQLJGZ',
                    '37i9dQZF1DX0h0QnLkMBl4',
                    '37i9dQZF1DX3Sp0P28SIer',
                    '37i9dQZF1DX0yEZaMOXna3',
                    '37i9dQZF1DXcagnSNtrGuJ',
                    '37i9dQZF1DXc6IFF23C9jj',
                    '37i9dQZF1DX4UkKv8ED8jp',
                    '37i9dQZF1DWYuGZUE4XQXm',
                    '37i9dQZF1DX3j9EYdzv2N9',
                    '37i9dQZF1DX1vSJnMeoy3V',
                    '37i9dQZF1DWWzQTBs5BHX9',
                    '37i9dQZF1DWTWdbR13PQYH',
                    '37i9dQZF1DXaW8fzPh9b08',
                    '37i9dQZF1DX0P7PzzKwEKl',
                    '37i9dQZF1DX9Ol4tZWPH6V',
                    '37i9dQZF1DWUZv12GM5cFk']

In [28]:
list_tops_per_year = list(map(playlist_features,top_ids_per_year))

In [34]:
df_tops_per_year = pd.concat(list_tops_per_year,
                            ignore_index = True)

In [37]:
df_without_nans = df_tops_per_year.dropna(axis = 1,
                                          how = "any")